# Using Time Series to more accurately predict wave heights

In [15]:
import utilities.misc as misc
import utilities.metrics as mtr
import utilities.ml_functions as ml
import pandas as pd
from datetime import datetime

In [25]:
# convert datetime to unix
df = pd.read_csv('clean_weather/valid_dates/m2_valid_dates.csv')
start_date = min(df['time'])
end_date = max(df['time'])

df['timestamps'] = (df['date'] + ' ' +df['time'])
# df['timestamps'] = datetime.strptime(df['timestamps'], '%m/%d/%y %H:%M:%S')
df['timestamps'] = pd.to_datetime(df['timestamps'])
df['timestamps'] = (df['timestamps'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')  

df.head()


,Unnamed: 0,time,station_id,AtmosphericPressure,WindDirection,WindSpeed,Gust,WaveHeight,WavePeriod,MeanWaveDirection,Hmax,SeaTemperature,date,timestamps
0,1953,00:00:00,M2,1020.0,330.0,2.92,3.9,0.6,4.0,NaN,NaN,8.6,2001-05-05,989020800
1,1955,01:00:00,M2,1020.8,330.0,4.09,5.8,0.4,4.0,NaN,NaN,8.6,2001-05-05,989024400
2,1957,02:00:00,M2,1020.8,350.0,4.09,5.8,0.4,4.0,NaN,NaN,8.7,2001-05-05,989028000
3,1959,03:00:00,M2,1021.2,350.0,1.95,5.8,0.4,4.0,NaN,NaN,8.7,2001-05-05,989031600
4,1961,04:00:00,M2,1021.4,30.0,2.92,5.8,0.4,4.0,NaN,NaN,8.7,2001-05-05,989035200


In [5]:
dates = pd.to_datetime(['2019-01-15 13:30:00'])
(dates - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')  
# Int64Index([1547559000], dtype='int64')

Int64Index([1547559000], dtype='int64')

In [ ]:
# plot data in time 


In [ ]:
# predict using daily seasonality